In [6]:
import json
import os

In [3]:
# input_dir = "files20232024"
input_dir = "../files"
output_file = "../nss.jsonl"

In [4]:
def preprocess_nss(content) -> tuple[str, str]:
            # content = content.replace("\n", " ")
            # content = content.replace("\t", " ")
            # content = content.replace("\r", " ")
            content = content.strip()
            try:
                name, content = content.split("\n", maxsplit=1)
                if name.startswith("Ej"):
                    name, content = content.split("\n", maxsplit=1)
            except ValueError:
                print(f"Document with missing title or text: {content}")
                return None, content
            return name, content

In [7]:
output_dataset = []
for c, file in enumerate(os.listdir(input_dir)):
    print(f"Opening {file}...")
    with open(f"{input_dir}/{file}") as f:
        content = f.read()
        # print(content)
    # break
    name, preprocessed_content = preprocess_nss(content)
    output_dataset.append({
        "id": c,
        "file": file,
        "name": name,
        # "title": name, # TODO?
        "contents": preprocessed_content
    })
    # print(F"Processed NAME: {name}")
    # print(f"Processed {preprocessed_content}")
    # break




Opening 452_2005.txt...
Opening 634_2005.txt...
Opening 3215_2015.txt...
Opening 1813_2009.txt...
Opening 3664_2018.txt...
Opening 3214_2015.txt...
Opening 635_2005.txt...
Opening 453_2005.txt...
Opening 1812_2009.txt...
Opening 3665_2018.txt...
Opening 1467_2008.txt...
Opening 988_2006.txt...
Opening 1601_2008.txt...
Opening 3876_2019.txt...
Opening 3513_2017.txt...
Opening 3097_2014.txt...
Opening 1600_2008.txt...
Opening 3877_2019.txt...
Opening 1466_2008.txt...
Opening 989_2006.txt...
Opening 3096_2014.txt...
Opening 3512_2017.txt...
Opening 4110_2021.txt...
Opening 384_2004.txt...
Opening 4111_2021.txt...
Opening 385_2004.txt...
Opening 1244_2007.txt...
Opening 812_2006.txt...
Opening 1022_2007.txt...
Opening 4470_2023.txt...
Opening 2242_2011.txt...
Opening 1023_2007.txt...
Opening 813_2006.txt...
Opening 1245_2007.txt...
Opening 2243_2011.txt...
Opening 4471_2023.txt...
Opening 2052_2010.txt...
Opening 4182_2021.txt...
Opening 4406_2022.txt...
Opening 170_2004.txt...
Opening 316

In [25]:
unchunked_output_file = "nss_unchunked.jsonl"
print(f"Saving to {unchunked_output_file}")
with open(unchunked_output_file, "w", encoding="utf-8") as f:
    # json.dump(output_dataset, f)
    for item in output_dataset:
        f.write(json.dumps(item) + "\n")
print(f"Saved to {unchunked_output_file}")

Saving to nss.jsonl
Saved to nss.jsonl


In [26]:
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize()

NameError: name 'text' is not defined

In [34]:
chunked_output_file = "../nss_chunked.jsonl"

In [35]:
chunked_output_dataset = []
global_counter = 0
for c, item in enumerate(output_dataset):
    content = item['contents']
    # sentences = sent_tokenize(content, language="czech")
    sentences = content.split("\n")
    sentences = [sentence for sentence in sentences if sentence.strip()]
    for sentence in sentences:
        chunked_output_dataset.append({
            "id": global_counter,
            "file": item['file'],
            "name": item['name'],
            "contents": sentence
        })
        global_counter += 1
    # break

In [36]:
with open(chunked_output_file, "w", encoding="utf-8") as f:
    for item in chunked_output_dataset:
        f.write(json.dumps(item) + "\n")

### sentence splitting by stanza

In [1]:
import stanza

# Download Czech models if not already downloaded
# stanza.download("cs")  # Only need to run this once

nlp = stanza.Pipeline(lang='cs', processors='tokenize,lemma,pos')

2025-03-30 14:49:31 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-03-30 14:49:31 INFO: Downloaded file to /Users/jakubkucera-sch/stanza_resources/resources.json
2025-03-30 14:49:31 WARNING: Language cs package default expects mwt, which has been added
2025-03-30 14:49:31 INFO: Loading these models for language: cs (Czech):
| Processor | Package      |
----------------------------
| tokenize  | pdt          |
| mwt       | pdt          |
| pos       | pdt_nocharlm |
| lemma     | pdt_nocharlm |

2025-03-30 14:49:31 INFO: Using device: cpu
2025-03-30 14:49:31 INFO: Loading: tokenize
2025-03-30 14:49:32 INFO: Loading: mwt
2025-03-30 14:49:32 INFO: Loading: pos
2025-03-30 14:49:33 INFO: Loading: lemma
2025-03-30 14:49:34 INFO: Done loading processors!


In [14]:
from tqdm import tqdm

stanza_chunked_output_dataset = []
global_counter = 0
# for c, item in tqdm(enumerate(output_dataset)):
for item in tqdm(output_dataset):
    content = item['contents']
    # print(f"content: {content}")
    doc = nlp(content)
    sentences = []
    for sentence in doc.sentences:
        sentences.append(sentence.text)
    # sentences = [sentence for sentence in sentences if sentence.strip()]
    for sentence in sentences:
        stanza_chunked_output_dataset.append({
            "id": global_counter,
            "file": item['file'],
            "name": item['name'],
            "contents": sentence
        })
        global_counter += 1
    # print("sentences")
    # for sentence in sentences:
    #     print(sentence)
    # break

 36%|███▌      | 1666/4649 [59:39<1:46:48,  2.15s/it]


KeyboardInterrupt: 

In [ ]:
with open("nss_stanza_chunked.jsonl", "w", encoding="utf-8") as f:
    for item in stanza_chunked_output_dataset:
        f.write(json.dumps(item) + "\n")